# Synthetic dataset generation for CNA demo

In [1]:
import numpy as np
import pandas as pd
import multianndata as mad
import scanpy as sc
np.random.seed(0)

In [2]:
# basic parameters
N = 50 # number of samples
G = 50 # number of genes
C = 200 # number of cells per sample
noise = 1

In [3]:
# assign covariates to samples
covs = pd.DataFrame(index=pd.Series(np.arange(N), name='id', dtype=int))
covs['case'] = [0]*int(N/2) + [1]*int(N/2)
covs['male'] = [0]*int(2*N/8) + [1]*int(2*N/8) + [0]*int(2*N/8) + [1]*(N - int(2*N/8)-int(2*N/8)-int(2*N/8))
covs['baseline'] = 1

In [4]:
# define GE profile of cell populations
H = np.zeros((3, G))
H[0,:int(G/2)] = 1
H[1,int(G/2):] = 1
H[2,:int(G/2)] = 1; H[2,:int(G/4)] = 2

# define cell ids for the cells in each sample
def getW(props):
    cell_ids = np.concatenate([np.array([i]*int(p*C)) for i,p in enumerate(props)])
    cell_ids = np.concatenate([cell_ids, np.array([len(props)]*int(C-len(cell_ids)))])
    W = np.zeros((int(C), len(props) + 1))
    for i in range(len(props) + 1):
        W[np.where(cell_ids == i)[0], i] = 1
    return W

props = np.array([
    [0.2, -0.2], #case
    [-0.2, 0], #male
    [0.5, 0.5]  #baseline
])
Ws = np.array([getW(c.dot(props)) for _, c in covs.iterrows()])

# create single-cell data
X = np.concatenate([W.dot(H) + noise*np.random.randn(C, G) for W in Ws])

# create multianndata object
d = mad.MultiAnnData(X=X,
                     obs=pd.DataFrame({'id':np.repeat(covs.index, C)}),
                     samplem=covs.drop(columns=['baseline']))
d.samplem['batch'] = np.tile(range(5), int(N/5))
d.var = pd.DataFrame({'gene':['gene'+str(i) for i in range(G)]})

/PHShome/lar24/anaconda3/lib/python3.7/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [6]:
sc.pp.neighbors(d)
sc.tl.umap(d)

In [7]:
d.write('data.h5ad')